In [ ]:
import numpy as np
import cv2 as cv
import os

import sys

sys.path.append("..")

import json
import matplotlib.pyplot as plt

# import apriltag
from scipy.spatial.transform import Rotation

import lac.params as params
from lac.utils.frames import (
    apply_transform,
    invert_transform_mat,
    get_cam_pose_rover,
    OPENCV_TO_CAMERA_PASSIVE,
)
from lac.perception.vision import FiducialLocalizer
from lac.utils.visualization import overlay_tag_detections
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%load_ext autoreload
%autoreload 2

# AprilTag detection


In [ ]:
# data_path = os.path.expanduser("../../output/lander_fiducials_right_light")
# data_path = os.path.expanduser("../../output/nav_agent_preset_0_seed_4")
data_path = os.path.expanduser("../../output/localization_preset_0_seed_4_1hz")
# data_path = os.path.expanduser("../../output/localization_preset_0_seed_4_1hz_maxres")
json_data = json.load(open(f"{data_path}/data_log.json"))
lander_pose = np.array(json_data["lander_pose_world"])
poses = np.array([frame["pose"] for frame in json_data["frames"]])

In [ ]:
camera_config = json_data["cameras"]

fid_localizer = FiducialLocalizer(camera_config)

In [ ]:
i = 60
CAM_NAME = "FrontLeft"
img = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)

detections = fid_localizer.detect(img)
overlay = overlay_tag_detections(img, detections)
plt.imshow(overlay)

In [ ]:
i = 2000
CAM_NAME = "Right"
img = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)

detections = fid_localizer.detect(img)
overlay = overlay_tag_detections(img, detections)
plt.imshow(overlay)

# Generate pose measurements


In [ ]:
fid_localizer.estimate_rover_pose(img, CAM_NAME, lander_pose)

In [ ]:
rover_pose = np.array(json_data["frames"][i]["pose"])
rover_pose

# Analyze Outliers


In [ ]:
# TODO: :disguised_face: # brotherrr

In [ ]:
# get the step ids of the camera frames
CAM_NAME = "Right"
img_names = os.listdir(os.path.join(data_path, CAM_NAME))
img_names = [int(name.split(".")[0]) for name in img_names]
img_names.sort()

In [ ]:
# run fiducial localizer on all images
fid_pose_estimates = {}
for i in img_names:
    fid_pose_estimates[i], _ = fid_localizer.estimate_rover_pose(
        cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE),
        CAM_NAME,
        lander_pose,
    )

In [ ]:
# compute state errors
state_errors = {}
idxes = np.arange(0, len(poses))
for pose_step_idx, frame in enumerate(poses):
    img_idx = pose_step_idx + 1  # account for a 1-indexed image name

    if img_idx not in img_names or not fid_pose_estimates[img_idx]:
        continue

    rover_pos = poses[pose_step_idx][:3, 3]
    rover_rpy = Rotation.from_matrix(poses[pose_step_idx][:3, :3]).as_euler("xyz", degrees=True)
    rover_state = np.hstack((rover_pos, rover_rpy))

    state_errors[pose_step_idx] = {}
    for tag_id, fpe in fid_pose_estimates[img_idx].items():
        rover_pos_fid = fpe[:3, 3]
        rover_rpy_fid = Rotation.from_matrix(fpe[:3, :3]).as_euler("xyz", degrees=True)
        rover_state_fid = np.hstack((rover_pos_fid, rover_rpy_fid))

        state_errors[pose_step_idx][tag_id] = rover_state_fid - rover_state


In [ ]:
# visualize state errors with errors in subplots on the y axis and frame index on the x axis
import random


def get_random_color():
    return f"rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})"


TAG_COLORS = {tag_id: get_random_color() for tag_id in params.TAG_LOCATIONS.keys()}

fig = make_subplots(rows=6, cols=1, subplot_titles=("X", "Y", "Z", "Roll", "Pitch", "Yaw"))

# plot the rover gt states
rover_poses = np.array([frame[:3, 3] for frame in poses])
rover_rpy = np.array(
    [Rotation.from_matrix(frame[:3, :3]).as_euler("xyz", degrees=True) for frame in poses]
)
for i in range(rover_poses.shape[1]):
    fig.add_trace(
        go.Scatter(
            x=idxes, y=rover_poses[:, i], mode="lines", name=f"Rover {i}", line=dict(color="black")
        ),
        row=i + 1,
        col=1,
    )
for i in range(rover_rpy.shape[1]):
    fig.add_trace(
        go.Scatter(
            x=idxes, y=rover_rpy[:, i], mode="lines", name=f"Rover {i}", line=dict(color="black")
        ),
        row=i + 4,
        col=1,
    )

# add horizontal line at 0 for each subplot
fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=3, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=4, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=5, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=6, col=1)

# plot the state errors
for pose_step_idx, errors in state_errors.items():
    if not errors:
        continue

    # for tag_id, error in errors.items():
    #     # if tag_id == 462 or tag_id == 37 or tag_id == 2 or tag_id == 1 or tag_id == 8 or tag_id == 9 or tag_id == 258 or tag_id == 5:
    #     #     continue
    #     color = TAG_COLORS[tag_id]
    #     fig.add_trace(go.Scatter(x=[frame_id], y=[error[0]], mode="markers", name=f"{tag_id}", marker=dict(color=color)), row=1, col=1)
    #     fig.add_trace(go.Scatter(x=[frame_id], y=[error[1]], mode="markers", name=f"{tag_id}", marker=dict(color=color)), row=1, col=2)
    #     fig.add_trace(go.Scatter(x=[frame_id], y=[error[2]], mode="markers", name=f"{tag_id}", marker=dict(color=color)), row=1, col=3)
    #     fig.add_trace(go.Scatter(x=[frame_id], y=[error[3]], mode="markers", name=f"{tag_id}", marker=dict(color=color)), row=2, col=1)
    #     fig.add_trace(go.Scatter(x=[frame_id], y=[error[4]], mode="markers", name=f"{tag_id}", marker=dict(color=color)), row=2, col=2)
    #     fig.add_trace(go.Scatter(x=[frame_id], y=[error[5]], mode="markers", name=f"{tag_id}", marker=dict(color=color)), row=2, col=3)

    errors = np.array(list(errors.values()))
    for i in range(errors.shape[1]):
        fig.add_trace(
            go.Box(x=[pose_step_idx] * len(errors), y=errors[:, i], name=f"{i}", showlegend=False),
            row=i + 1,
            col=1,
        )

# change display to x-unified
fig.update_layout(hovermode="x unified")
fig.update_xaxes(title_text="Frame Index")
fig.update_layout(height=1600, width=1800, title_text="State Errors")
fig.show()

In [ ]:
# analyze specific frames
pose_step_idxes = np.arange(2179, 2279, 20)

for ind in pose_step_idxes:
    img = cv.imread(os.path.join(data_path, CAM_NAME, f"{ind + 1}.png"), cv.IMREAD_GRAYSCALE)

    detections = fid_localizer.detect(img)
    overlay = overlay_tag_detections(img, detections)
    plt.imshow(overlay)
    plt.show()

    print(state_errors[ind])